<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/Bits-To-Models-From-Orbit/blob/main/Hour_4_Tutorial_to_Satellite_Imagery_and_Remote_Sensing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Image Analysis in Google Earth Engine - 1 Hour Tutorial
Introduction (5 minutes) **bold text**

Briefly introduce Google Earth Engine as a cloud-based platform for planetary-scale geospatial analysis using satellite imagery and other datasets

Explain the goal is to provide an overview of key image analysis techniques in Earth Engine in an hour

Cover the main topics that will be addressed:

Image preprocessing

Digital image processing

Image classification

Discuss common preprocessing like geometric and radiometric correction

In [ ]:
# Import and initialize Earth Engine
import ee
ee.Initialize()

# Load an image
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Get projection info
print(image.projection())

# Reproject image
reprojected = image.reproject('EPSG:4326')

Show image registration

In [ ]:
# Load two images
image1 = ee.Image('IMAGE1')
image2 = ee.Image('IMAGE2')

# Register image2 to image1
registered = image2.register(referenceImage=image1)

**Image Preprocessing (15 minutes)**

Discuss common preprocessing steps like geometric and radiometric correction

Show how to find image metadata like projection info in Earth Engine

Demonstrate reprojection of an image from its native projection to another standard projection

Illustrate image registration to align images using an image pair

Introduce spectral indices like NDVI and show how to calculate them

Demonstrate cloud and shadow removal techniques

In [ ]:
# Load an image
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Compute NDVI
ndvi = image.normalizedDifference(['B5','B4'])

# Display NDVI
Map.addLayer(ndvi)

**Digital Image Processing (15 minutes)**

Explain the goal of image enhancement and categories like contrast manipulation, spatial filtering, etc.
Show grey level thresholding to segment water pixels from an image

Illustrate level slicing to highlight different water content based on NDWI

Demonstrate spatial filtering using convolution with low-pass and high-pass kernels

Show Canny edge detection and Hough line detection
Illustrate pan-sharpening by converting RGB bands to HSV space

Show texture analysis using entropy filter on high resolution imagery

In [ ]:
# Load image
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Threshold NIR band
watermask = image.select('B5').gt(0.1)

# Mask water pixels
masked = image.updateMask(watermask)

In [ ]:
# Compute NDWI
ndwi = image.normalizedDifference(['B3','B5'])

# Slice NDWI into classes
classes = ndwi.gt(-0.1).add(1).updateMask(ndwi.lt(0.1)).add(2)

# Display slices
Map.addLayer(classes, {min: 1, max: 3, palette: ['00FFFF', '0000FF']})

In [ ]:
# Load image
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Define kernels
lowpass = ee.Kernel.square(radius=5, normalize=True)
highpass = ee.Kernel.square(radius=5, normalize=False)

# Filter images
smooth = image.convolve(lowpass)
sharp = image.convolve(highpass)

In [ ]:
# Load image
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Canny edge detection
canny = ee.Algorithms.CannyEdgeDetector(image, 0.7)

# Hough transform
hough = ee.Algorithms.HoughTransform(canny, 256, 50, 10)

In [ ]:
# Load image
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Convert RGB to HSV
hsv = image.select('B[1-3]').rgbToHsv()

# Merge with panchromatic band
merged = ee.Image.cat(hsv, image.select('B8'))

# Convert back to RGB
sharpened = merged.hsvToRgb()

In [ ]:
# Load NAIP image
image = ee.Image('USDA/NAIP/DOQQ/m_3712213_sw_10_1_20140613')

# Compute entropy
entropy = image.entropy(ee.Kernel.square(7))

**Image Classification (20 minutes)**

Introduce the concept of supervised and unsupervised classification

Show example of unsupervised classification using K-means clustering

Demonstrate supervised classification with random forest algorithm

Discuss need for training data and strategies for collecting suitable training data

Illustrate accuracy assessment after classification to quantify errors

Show classification refinement techniques like sieving and clumping

In [ ]:
# Load image
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Cluster into 5 classes
clusters = image.cluster(numberOfClusters=5)

In [ ]:
# Load training data
training = ee.FeatureCollection('TRAINING_DATA')

# Train random forest model
rf = ee.Classifier.smileRandomForest(10) \
  .train(training, 'class', bands)

# Classify image
classified = image.select(bands).classify(rf)

In [ ]:
# Load test data
test = ee.FeatureCollection('TEST_DATA')

# Assess accuracy
testAccuracy = classified.errorMatrix(test, 'class', 'classification')
print('Accuracy:', testAccuracy.accuracy())

In [ ]:
# Sieve to remove small clusters
sieved = classified.sieve(5)

# Clump classes
clumped = sieved.clump()


Conclusion (5 minutes) **bold text**

Briefly summarize key topics covered in the tutorial

Point to documentation and other resources for learning more

Invite questions and discussion